# Function Calling
- Function Calling은 OpenAI API에서 특정 함수 정의(JSON schema 기반)를 주고, 모델이 해당 함수를 호출하는 형식의 응답을 생성하게 하는 기능입니다.
- 챗봇, 에이전트(Agent), 도구(Tool) 사용 등 다양한 자동화 및 플러그인 연동에 활용됩니다.

- 지원 모델 gpt-3-turbo, gpt-4, gpt-4-turbo  (2025.1 현재)

- 모델이 우리의 코드를 호출하도록 해서 우리의 함수들을 모델이 호출할수 있도록 하거나
- 모델이 우리가 원하는 특정 모양과 형식의 output 을 갖도록 강제할수 있다.

- https://platform.openai.com/docs/guides/function-calling?api-mode=chat





In [1]:
import os 

from dotenv import load_dotenv
load_dotenv()

print(f'OPENAI_API_KEY={os.getenv("OPENAI_API_KEY")[:20]}')

OPENAI_API_KEY=sk-proj-iKU13YeoxNgF


In [2]:
from langchain_openai.chat_models.base import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate

In [4]:
llm = ChatOpenAI(
    temperature=0.1, 
)

prompt = PromptTemplate.from_template("Who is the weather in {city}")  # Who 맞다.

chain = prompt | llm

response = chain.invoke({
    "city": "rome"
})

print(response.content)


I'm sorry, I am not able to provide real-time weather updates. I recommend checking a reliable weather website or app for the most up-to-date information on the weather in Rome.


In [ ]:
# 실시간으로 날씨를 가져올수 없다.
# 하지만!  이제 함수가 있다고 생각해보자.  실시간 데이터를 가져올 수 잇는 함수 말이다!
#  함수에 필요한 정보는 아마 '장소' 정보 뿐 일거다.  (위도, 경도..)

# ↓여기에 함수가 있다고 가정하자

In [16]:
# ↓ 이러한 함수가 있다고 가정하자
#   위도,경도를 주면 실시간 날씨 정보를 리턴하는 함수..
def get_weather(lon, lat):
    print(f"call an api...lon:{lon}, lat:{lat}")

# ↓ LLM 에게 우리에게는 '이러이러한 동작' 을 해주는 함수가 있다고 알려주어야 한다
#   함수를 JSON schema 로 정의!

# 함수의 스키마

In [5]:
function_schema = {
    "name": "get_weather",  # 함수의 이름

    # description : 함수가 무슨 일을 하는지 기술
    # ↓'위도와 경도를 받아서 특정장소의 날씨정보를 가져오는 함수"
    "description": "function that takes longitude and latitude to find the weather of a place",  

    # 파라미터 기술
    "parameters": {
        "type": "object",
        "properties": {
            # 경도
            "lon": {"type": "string", "description": "The longitude coordinate",},
            # 위도
            "lat": {"type": "string","description": "The latitude coordinate",},
        },
    },

    # 필수 사항 기술
    "required": ["lon", 'lat'],  
}

# ChatOpenAI 에 함수 전달하기

In [8]:
llm = ChatOpenAI(
    temperature=0.1,
).bind(  # ChatOpenAI 에게 전달할 인수를 추가해줄수 있다.
    functions=[  # 원하는 만큼 많은 함수를 전달해 줄수 있다
        function_schema,   # 준비한 schema 를 전달!
    ],

    # ↓기본적으로 모델이 특정 함수를 사용하도록 '강제'하거나, 
    #   모델이 함수를 사용하도록 하거나 그냥 답변을 할 수 있도록 모델 스스로 선택하게 할수 있다.

    # 모델을 강제로 함수를 사용하도록 하려면! 아래와 같이 해야 한다
    # function_call={  
    #     "name": "get_weather",  # <-- get_weather 함수를 반드시 사용하라!        
    # },
    
    # AI에 선택권을 주어 필요에 따라 선택하여 함수를 사용하게 함.
    function_call='auto',
)

In [9]:
chain = prompt | llm

response = chain.invoke({
    "city": "rome"
})

response


AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"lon":"12.4964","lat":"41.9028"}', 'name': 'get_weather'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 74, 'total_tokens': 98, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BwSWX6dM7lLk5NvGuOcLJZDax9TaH', 'finish_reason': 'function_call', 'logprobs': None}, id='run--4ad219bc-50fc-4b4a-bfec-45a81c7e3e11-0', usage_metadata={'input_tokens': 74, 'output_tokens': 24, 'total_tokens': 98, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [10]:
response.additional_kwargs

{'function_call': {'arguments': '{"lon":"12.4964","lat":"41.9028"}',
  'name': 'get_weather'},
 'refusal': None}

In [11]:
response.additional_kwargs['function_call']

{'arguments': '{"lon":"12.4964","lat":"41.9028"}', 'name': 'get_weather'}

In [12]:
response.additional_kwargs['function_call']['arguments']

'{"lon":"12.4964","lat":"41.9028"}'

In [13]:
import json

r = json.loads(response.additional_kwargs['function_call']['arguments'])

r

{'lon': '12.4964', 'lat': '41.9028'}

In [14]:
r['lon'], r['lat']

('12.4964', '41.9028')

In [17]:
get_weather(r['lon'], r['lat'])

call an api...lon:12.4964, lat:41.9028


In [18]:
get_weather(**r)  # argument unpacking

call an api...lon:12.4964, lat:41.9028


In [20]:
from langchain_openai.chat_models.base import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
import json

def get_weather(lon, lat):
    print(f"call an api...lon:{lon}, lat{lat}")  

function_schema = {
    "name": "get_weather",
    "description": "function that takes longitude and latitude to find the weather of a place",  
    
    "parameters": { 
        "type": "object",
        "properties": {
            "lon": {
                "type": "string",
                "description": "The longitude coordinate",
            },
            "lat": {
                "type": "string",
                "description": "The latitude coordinate",
            },
        },
    },

    "required": ["lon", "lat"],
}    


llm = ChatOpenAI(
    temperature=0.1,
).bind(
    functions=[
        function_schema
    ],
    function_call="auto"
)

prompt = PromptTemplate.from_template("Who is the weather in {city}")

chain = prompt | llm

response = chain.invoke({
    "city": "Seoul"
})

r = json.loads(response.additional_kwargs['function_call']['arguments'])
get_weather(**r)

call an api...lon:126.9779, lat37.5665


# 퀴즈 생성하기 스키마

In [21]:
function_schema = {
    "name": "create_quiz",
    # ↓ 완전히 지어낸 함수다!
    "description": "function that takes a list of questions and answers and returns a quiz",  

    "parameters": {
        "type": "object",
        "properties": {
# ↓ 우라기 원하는 답변의 스키마는
 
# 바로 이 형태를 정의하는 거다. 
# {
#   "questions":[
#      0:{
#        "question":"What … in the story?"
#        "answers":[
#           0:{
#             "answer":"John"
#             "correct":false
#           }
#           ... 
#        ]
#      }
#      ...
# }     

            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": { "type": "string"},
                                    "correct": { "type": "boolean"},
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"]
                },
            },
        },        
    },
    "required": ["questions"],  
}

In [23]:
llm = ChatOpenAI(
    temperature=0.1
).bind(
    functions=[function_schema],
    function_call={
        "name": "create_quiz",
    }
)

prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm
response = chain.invoke({
    "city": "Rome"
})

response = json.loads(response.additional_kwargs['function_call']['arguments'])

response

{'questions': [{'question': 'What is the capital city of Italy?',
   'answers': [{'answer': 'Rome', 'correct': True},
    {'answer': 'Milan', 'correct': False},
    {'answer': 'Florence', 'correct': False}]},
  {'question': 'Which ancient Roman structure is known for its gladiator contests?',
   'answers': [{'answer': 'Colosseum', 'correct': True},
    {'answer': 'Pantheon', 'correct': False},
    {'answer': 'Trevi Fountain', 'correct': False}]},
  {'question': 'Who was the first Roman Emperor?',
   'answers': [{'answer': 'Julius Caesar', 'correct': False},
    {'answer': 'Augustus', 'correct': True},
    {'answer': 'Nero', 'correct': False}]},
  {'question': 'What river runs through the city of Rome?',
   'answers': [{'answer': 'Tiber River', 'correct': True},
    {'answer': 'Arno River', 'correct': False},
    {'answer': 'Po River', 'correct': False}]},
  {'question': 'Which famous Roman general crossed the Alps with elephants?',
   'answers': [{'answer': 'Julius Caesar', 'correct': 

In [ ]:
# 원하는 형태의 출력을 하려면?
#  example + output parser 와  function calling 두가지중 선택해볼수 있다.